In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

In [2]:
# Load your datasets with an alternate encoding
try:
    removal_sites = pd.read_csv('Removal_Sites.csv', encoding='utf-8')
except UnicodeDecodeError:
    removal_sites = pd.read_csv('Removal_Sites.csv', encoding='ISO-8859-1')  # or 'latin1'

try:
    disposal_sites = pd.read_csv('Disposal_Sites.csv', encoding='utf-8')
except UnicodeDecodeError:
    disposal_sites = pd.read_csv('Disposal_Sites.csv', encoding='ISO-8859-1')

try:
    dist_proper = pd.read_csv('Removal_to_Dispoal_Distance.csv', encoding='utf-8')
except UnicodeDecodeError:
    dist_proper = pd.read_csv('Removal_to_Dispoal_Distance.csv', encoding='ISO-8859-1')
    
try:
    dist_sites = pd.read_csv('Removal_to_Removal_Distance.csv', encoding='utf-8')
except UnicodeDecodeError:
    dist_sites = pd.read_csv('Removal_to_Removal_Distance.csv',encoding='ISO-8859-1')

## First Priority as Hub

In [3]:
import gurobipy as gp
from gurobipy import GRB

# Create a Gurobi model
model = gp.Model('SnowRemoval')
model.Params.OutputFlag = 0
# Set the NonConvex parameter
model.setParam('NonConvex', 2)

# Define the hub (assuming the hub is one of the removal sites)
hub_id = 'VMA-109'  # Replace with the actual ID of the hub

# Decision variables
removal_ids = removal_sites['NomSecteur'].tolist()
disposal_ids = disposal_sites['NomDepot'].tolist()
X = model.addVars(removal_ids, disposal_ids, vtype=GRB.BINARY, name="X")
Y = model.addVars(removal_ids, vtype=GRB.CONTINUOUS, name="Y")
Z = model.addVars(removal_ids, removal_ids, vtype=GRB.BINARY, name="Z")  # Auxiliary variables

# Pre-calculate cost and round-trip distances
cost_distance_dict = {}

for i in removal_ids:
    for j in disposal_ids:
        # Get disposal cost and convert to float
        disposal_cost = float(disposal_sites[disposal_sites['NomDepot'] == j]['Cost ($/m3)'].iloc[0]) if disposal_sites[disposal_sites['NomDepot'] == j]['Cost ($/m3)'].iloc[0] else 0.0

        # Get distance to disposal site and convert to float
        distance_to_disposal = float(dist_proper[(dist_proper['PickupSector'] == i) & (dist_proper['DisposalSite'] == j)][' Distance (km)'].iloc[0]) if not dist_proper[(dist_proper['PickupSector'] == i) & (dist_proper['DisposalSite'] == j)][' Distance (km)'].empty else 0.0
        
        # Get distance from disposal site to hub and convert to float
        distance_to_hub = float(dist_proper[(dist_proper['PickupSector'] == hub_id) & (dist_proper['DisposalSite'] == j)][' Distance (km)'].iloc[0]) if not dist_proper[(dist_proper['PickupSector'] == hub_id) & (dist_proper['DisposalSite'] == j)][' Distance (km)'].empty else 0.0

        # Get distance from hub to removal site i (assuming symmetric distances)
        distance_from_hub_to_i = float(dist_sites[(dist_sites['Removal Site 1'] == hub_id) & (dist_sites['Removal Site 2'] == i)]['Distance'].iloc[0]) if not dist_sites[(dist_sites['Removal Site 1'] == hub_id) & (dist_sites['Removal Site 2'] == i)].empty else 0.0

        # Total round-trip distance
        total_distance = distance_from_hub_to_i + distance_to_disposal + distance_to_hub

        # Store in dictionary
        cost_distance_dict[(i, j)] = (disposal_cost, total_distance)

# Objective function with penalty for priority
penalty_factor = 100000
objective = gp.quicksum(
    (Y[i] * cost_distance_dict[(i, j)][0] + Y[i]*(cost_distance_dict[(i, j)][1] * 0.1395 + 0.513)) * X[i, j]
    for i in removal_ids for j in disposal_ids)

# Adding penalty logic for priorities
for i in removal_ids:
    for k in removal_ids:
        if i != k:
            priority_diff = float(removal_sites[removal_sites['NomSecteur'] == k]['Priority'].iloc[0]) - \
                            float(removal_sites[removal_sites['NomSecteur'] == i]['Priority'].iloc[0])
            objective += penalty_factor * Z[i, k] * max(0, priority_diff)

model.setObjective(objective, GRB.MINIMIZE)

# Constraints
# 1. Capacity of disposal sites
for j in disposal_ids:
    model.addConstr(gp.quicksum(Y[i] * X[i, j] for i in removal_ids) <= float(disposal_sites[disposal_sites['NomDepot'] == j]['Capacite m3'].iloc[0]))

# 2. Completeness of snow removal
for i in removal_ids:
    model.addConstr(gp.quicksum(X[i, j] for j in disposal_ids) == 1)

# 3. Total snow transported from each removal site equals its snow volume
for i in removal_ids:
    total_snow_volume = float(removal_sites[removal_sites['NomSecteur'] == i]['Volume m3'].iloc[0])
    model.addConstr(gp.quicksum(Y[i] * X[i, j] for j in disposal_ids) == total_snow_volume)

# Additional constraints for priority ordering
for i in removal_ids:
    for k in removal_ids:
        if i != k:
            model.addConstr(gp.quicksum(X[i, j] for j in disposal_ids) - gp.quicksum(X[k, j] for j in disposal_ids) <= Z[i, k])

# Optimize the model
model.optimize()

# Check if an optimal solution was found
if model.status == GRB.OPTIMAL:
    # Print the total cost
    total_cost = model.ObjVal
    print(f"Total Cost of Snow Removal: ${total_cost:.2f}")

    # Print the amount of snow transported for each route
    print("\nSnow Transported for Each Route:")
    for i in removal_ids:
        for j in disposal_ids:
            if X[i, j].X > 0.5:  # If this route is used
                snow_transported = Y[i].X
                print(f"Removal site {i} -> Disposal site {j}: {snow_transported} m³")
else:
    print("No optimal solution found.")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-30
Total Cost of Snow Removal: $57035642.81

Snow Transported for Each Route:
Removal site VMA-109 -> Disposal site Fullum (VMA): 93274.51 m³
Removal site VMA-307 -> Disposal site Fullum (VMA): 82691.13 m³
Removal site VMA-306 -> Disposal site Fullum (VMA): 96229.11 m³
Removal site VMA-111 -> Disposal site Fullum (VMA): 85847.51 m³
Removal site VMA-110 -> Disposal site Riverside (VMA): 127483.61 m³
Removal site PMR-101 -> Disposal site Fullum (VMA): 158592.4 m³
Removal site PMR-102 -> Disposal site Fullum (VMA): 144354.73 m³
Removal site S-O-104 -> Disposal site Riverside (VMA): 136883.4 m³
Removal site PMR-204 -> Disposal site Iberville (PMR): 80600.83 m³
Removal site MHM-109 -> Disposal site Iberville (PMR): 126720.19 m³
Removal site PMR-203 -> Disposal site Fullum (VMA): 97228.93 m³
Removal site RPP-201 -> Disposal site Fullum (VMA): 116858.26 m³
Removal site MHM-209 -> Disposal site Iberville (P

## Least Prioirty as hub

In [6]:
import gurobipy as gp
from gurobipy import GRB

# Create a Gurobi model
model = gp.Model('SnowRemoval')
model.Params.OutputFlag = 0
# Set the NonConvex parameter
model.setParam('NonConvex', 2)

# Define the hub (assuming the hub is one of the removal sites)
hub_id = 'IBI-301'  # Replace with the actual ID of the hub

# Decision variables
removal_ids = removal_sites['NomSecteur'].tolist()
disposal_ids = disposal_sites['NomDepot'].tolist()
X = model.addVars(removal_ids, disposal_ids, vtype=GRB.BINARY, name="X")
Y = model.addVars(removal_ids, vtype=GRB.CONTINUOUS, name="Y")
Z = model.addVars(removal_ids, removal_ids, vtype=GRB.BINARY, name="Z")  # Auxiliary variables

# Pre-calculate cost and round-trip distances
cost_distance_dict = {}

for i in removal_ids:
    for j in disposal_ids:
        # Get disposal cost and convert to float
        disposal_cost = float(disposal_sites[disposal_sites['NomDepot'] == j]['Cost ($/m3)'].iloc[0]) if disposal_sites[disposal_sites['NomDepot'] == j]['Cost ($/m3)'].iloc[0] else 0.0

        # Get distance to disposal site and convert to float
        distance_to_disposal = float(dist_proper[(dist_proper['PickupSector'] == i) & (dist_proper['DisposalSite'] == j)][' Distance (km)'].iloc[0]) if not dist_proper[(dist_proper['PickupSector'] == i) & (dist_proper['DisposalSite'] == j)][' Distance (km)'].empty else 0.0
        
        # Get distance from disposal site to hub and convert to float
        distance_to_hub = float(dist_proper[(dist_proper['PickupSector'] == hub_id) & (dist_proper['DisposalSite'] == j)][' Distance (km)'].iloc[0]) if not dist_proper[(dist_proper['PickupSector'] == hub_id) & (dist_proper['DisposalSite'] == j)][' Distance (km)'].empty else 0.0

        # Get distance from hub to removal site i (assuming symmetric distances)
        distance_from_hub_to_i = float(dist_sites[(dist_sites['Removal Site 1'] == hub_id) & (dist_sites['Removal Site 2'] == i)]['Distance'].iloc[0]) if not dist_sites[(dist_sites['Removal Site 1'] == hub_id) & (dist_sites['Removal Site 2'] == i)].empty else 0.0

        # Total round-trip distance
        total_distance = distance_from_hub_to_i + distance_to_disposal + distance_to_hub

        # Store in dictionary
        cost_distance_dict[(i, j)] = (disposal_cost, total_distance)

# Objective function with penalty for priority
penalty_factor = 100000
objective = gp.quicksum(
    (Y[i] * cost_distance_dict[(i, j)][0] + Y[i]*(cost_distance_dict[(i, j)][1] * 0.1395 + 0.513)) * X[i, j]
    for i in removal_ids for j in disposal_ids)

# Adding penalty logic for priorities
for i in removal_ids:
    for k in removal_ids:
        if i != k:
            priority_diff = float(removal_sites[removal_sites['NomSecteur'] == k]['Priority'].iloc[0]) - \
                            float(removal_sites[removal_sites['NomSecteur'] == i]['Priority'].iloc[0])
            objective += penalty_factor * Z[i, k] * max(0, priority_diff)

model.setObjective(objective, GRB.MINIMIZE)

# Constraints
# 1. Capacity of disposal sites
for j in disposal_ids:
    model.addConstr(gp.quicksum(Y[i] * X[i, j] for i in removal_ids) <= float(disposal_sites[disposal_sites['NomDepot'] == j]['Capacite m3'].iloc[0]))

# 2. Completeness of snow removal
for i in removal_ids:
    model.addConstr(gp.quicksum(X[i, j] for j in disposal_ids) == 1)

# 3. Total snow transported from each removal site equals its snow volume
for i in removal_ids:
    total_snow_volume = float(removal_sites[removal_sites['NomSecteur'] == i]['Volume m3'].iloc[0])
    model.addConstr(gp.quicksum(Y[i] * X[i, j] for j in disposal_ids) == total_snow_volume)

# Additional constraints for priority ordering
for i in removal_ids:
    for k in removal_ids:
        if i != k:
            model.addConstr(gp.quicksum(X[i, j] for j in disposal_ids) - gp.quicksum(X[k, j] for j in disposal_ids) <= Z[i, k])

# Optimize the model
model.optimize()

# Check if an optimal solution was found
if model.status == GRB.OPTIMAL:
    # Print the total cost
    total_cost = model.ObjVal
    print(f"Total Cost of Snow Removal: ${total_cost:.2f}")

    # Print the amount of snow transported for each route
    print("\nSnow Transported for Each Route:")
    for i in removal_ids:
        for j in disposal_ids:
            if X[i, j].X > 0.5:  # If this route is used
                snow_transported = Y[i].X
                print(f"Removal site {i} -> Disposal site {j}: {snow_transported} m³")
else:
    print("No optimal solution found.")

Total Cost of Snow Removal: $121770460.70

Snow Transported for Each Route:
Removal site VMA-109 -> Disposal site Saint-Pierre (LAC): 93274.51 m³
Removal site VMA-307 -> Disposal site SauvÃ© (AHU): 82691.13 m³
Removal site VMA-306 -> Disposal site SauvÃ© (AHU): 96229.11 m³
Removal site VMA-111 -> Disposal site SauvÃ© (AHU): 85847.51 m³
Removal site VMA-110 -> Disposal site Saint-Pierre (LAC): 127483.61 m³
Removal site PMR-101 -> Disposal site SauvÃ© (AHU): 158592.4 m³
Removal site PMR-102 -> Disposal site SauvÃ© (AHU): 144354.73 m³
Removal site S-O-104 -> Disposal site Saint-Pierre (LAC): 136883.4 m³
Removal site PMR-204 -> Disposal site Millen (AHU): 80600.83 m³
Removal site MHM-109 -> Disposal site Millen (AHU): 126720.19 m³
Removal site PMR-203 -> Disposal site SauvÃ© (AHU): 97228.93 m³
Removal site RPP-201 -> Disposal site Millen (AHU): 116858.26 m³
Removal site MHM-209 -> Disposal site Millen (AHU): 124186.67 m³
Removal site MHM-210 -> Disposal site Millen (AHU): 156288.64 m³
Remo

## Middle priority as hub

In [7]:
import gurobipy as gp
from gurobipy import GRB

# Create a Gurobi model
model = gp.Model('SnowRemoval')
model.Params.OutputFlag = 0
# Set the NonConvex parameter
model.setParam('NonConvex', 2)

# Define the hub (assuming the hub is one of the removal sites)
hub_id = 'LAS-326'  # Replace with the actual ID of the hub 

# Decision variables
removal_ids = removal_sites['NomSecteur'].tolist()
disposal_ids = disposal_sites['NomDepot'].tolist()
X = model.addVars(removal_ids, disposal_ids, vtype=GRB.BINARY, name="X")
Y = model.addVars(removal_ids, vtype=GRB.CONTINUOUS, name="Y")
Z = model.addVars(removal_ids, removal_ids, vtype=GRB.BINARY, name="Z")  # Auxiliary variables

# Pre-calculate cost and round-trip distances
cost_distance_dict = {}

for i in removal_ids:
    for j in disposal_ids:
        # Get disposal cost and convert to float
        disposal_cost = float(disposal_sites[disposal_sites['NomDepot'] == j]['Cost ($/m3)'].iloc[0]) if disposal_sites[disposal_sites['NomDepot'] == j]['Cost ($/m3)'].iloc[0] else 0.0

        # Get distance to disposal site and convert to float
        distance_to_disposal = float(dist_proper[(dist_proper['PickupSector'] == i) & (dist_proper['DisposalSite'] == j)][' Distance (km)'].iloc[0]) if not dist_proper[(dist_proper['PickupSector'] == i) & (dist_proper['DisposalSite'] == j)][' Distance (km)'].empty else 0.0
        
        # Get distance from disposal site to hub and convert to float
        distance_to_hub = float(dist_proper[(dist_proper['PickupSector'] == hub_id) & (dist_proper['DisposalSite'] == j)][' Distance (km)'].iloc[0]) if not dist_proper[(dist_proper['PickupSector'] == hub_id) & (dist_proper['DisposalSite'] == j)][' Distance (km)'].empty else 0.0

        # Get distance from hub to removal site i (assuming symmetric distances)
        distance_from_hub_to_i = float(dist_sites[(dist_sites['Removal Site 1'] == hub_id) & (dist_sites['Removal Site 2'] == i)]['Distance'].iloc[0]) if not dist_sites[(dist_sites['Removal Site 1'] == hub_id) & (dist_sites['Removal Site 2'] == i)].empty else 0.0

        # Total round-trip distance
        total_distance = distance_from_hub_to_i + distance_to_disposal + distance_to_hub

        # Store in dictionary
        cost_distance_dict[(i, j)] = (disposal_cost, total_distance)

# Objective function with penalty for priority
penalty_factor = 100000
objective = gp.quicksum(
    (Y[i] * cost_distance_dict[(i, j)][0] + Y[i]*(cost_distance_dict[(i, j)][1] * 0.1395 + 0.513)) * X[i, j]
    for i in removal_ids for j in disposal_ids)

# Adding penalty logic for priorities
for i in removal_ids:
    for k in removal_ids:
        if i != k:
            priority_diff = float(removal_sites[removal_sites['NomSecteur'] == k]['Priority'].iloc[0]) - \
                            float(removal_sites[removal_sites['NomSecteur'] == i]['Priority'].iloc[0])
            objective += penalty_factor * Z[i, k] * max(0, priority_diff)

model.setObjective(objective, GRB.MINIMIZE)

# Constraints
# 1. Capacity of disposal sites
for j in disposal_ids:
    model.addConstr(gp.quicksum(Y[i] * X[i, j] for i in removal_ids) <= float(disposal_sites[disposal_sites['NomDepot'] == j]['Capacite m3'].iloc[0]))

# 2. Completeness of snow removal
for i in removal_ids:
    model.addConstr(gp.quicksum(X[i, j] for j in disposal_ids) == 1)

# 3. Total snow transported from each removal site equals its snow volume
for i in removal_ids:
    total_snow_volume = float(removal_sites[removal_sites['NomSecteur'] == i]['Volume m3'].iloc[0])
    model.addConstr(gp.quicksum(Y[i] * X[i, j] for j in disposal_ids) == total_snow_volume)

# Additional constraints for priority ordering
for i in removal_ids:
    for k in removal_ids:
        if i != k:
            model.addConstr(gp.quicksum(X[i, j] for j in disposal_ids) - gp.quicksum(X[k, j] for j in disposal_ids) <= Z[i, k])

# Optimize the model
model.optimize()

# Check if an optimal solution was found
if model.status == GRB.OPTIMAL:
    # Print the total cost
    total_cost = model.ObjVal
    print(f"Total Cost of Snow Removal: ${total_cost:.2f}")

    # Print the amount of snow transported for each route
    print("\nSnow Transported for Each Route:")
    for i in removal_ids:
        for j in disposal_ids:
            if X[i, j].X > 0.5:  # If this route is used
                snow_transported = Y[i].X
                print(f"Removal site {i} -> Disposal site {j}: {snow_transported} m³")
else:
    print("No optimal solution found.")

Total Cost of Snow Removal: $53226555.05

Snow Transported for Each Route:
Removal site VMA-109 -> Disposal site Fullum (VMA): 93274.51 m³
Removal site VMA-307 -> Disposal site Fullum (VMA): 82691.13 m³
Removal site VMA-306 -> Disposal site Iberville (PMR): 96229.11 m³
Removal site VMA-111 -> Disposal site Iberville (PMR): 85847.51 m³
Removal site VMA-110 -> Disposal site Riverside (VMA): 127483.61 m³
Removal site PMR-101 -> Disposal site Iberville (PMR): 158592.4 m³
Removal site PMR-102 -> Disposal site Riverside (VMA): 144354.73 m³
Removal site S-O-104 -> Disposal site Butler (S-O): 136883.4 m³
Removal site PMR-204 -> Disposal site Iberville (PMR): 80600.83 m³
Removal site MHM-109 -> Disposal site Iberville (PMR): 126720.19 m³
Removal site PMR-203 -> Disposal site Iberville (PMR): 97228.93 m³
Removal site RPP-201 -> Disposal site Millen (AHU): 116858.26 m³
Removal site MHM-209 -> Disposal site De la Salle (MHM): 124186.67 m³
Removal site MHM-210 -> Disposal site De la Salle (MHM): 15

## Finding the optimal hub

Code takes 1.5 hours to run - the optimal hub is VSP-206​. We are running the code for 'VSP-206' as hub at the end.

In [ ]:

# # Initialize an empty DataFrame to store results
# results_df = pd.DataFrame(columns=['Hub', 'TotalCost'])
# route_details_df = pd.DataFrame(columns=['Hub', 'RemovalSite', 'DisposalSite', 'SnowTransported'])

# # Create a Gurobi model
# model = gp.Model('SnowRemoval')

# # Set the NonConvex parameter
# model.setParam('NonConvex', 2)

# # Iterate over each potential hub
# for potential_hub in removal_sites['NomSecteur'].tolist():
#     # Set the hub_id to the current potential hub
#     hub_id = potential_hub

#     # Decision variables
#     removal_ids = removal_sites['NomSecteur'].tolist()
#     disposal_ids = disposal_sites['NomDepot'].tolist()
#     X = model.addVars(removal_ids, disposal_ids, vtype=GRB.BINARY, name="X")
#     Y = model.addVars(removal_ids, vtype=GRB.CONTINUOUS, name="Y")
#     Z = model.addVars(removal_ids, removal_ids, vtype=GRB.BINARY, name="Z")  # Auxiliary variables

#     # Pre-calculate cost and round-trip distances
#     cost_distance_dict = {}

#     for i in removal_ids:
#         for j in disposal_ids:
#             # Get disposal cost and convert to float
#             disposal_cost = float(disposal_sites[disposal_sites['NomDepot'] == j]['Cost ($/m3)'].iloc[0]) if disposal_sites[disposal_sites['NomDepot'] == j]['Cost ($/m3)'].iloc[0] else 0.0

#             # Get distance to disposal site and convert to float
#             distance_to_disposal = float(dist_proper[(dist_proper['PickupSector'] == i) & (dist_proper['DisposalSite'] == j)][' Distance (km)'].iloc[0]) if not dist_proper[(dist_proper['PickupSector'] == i) & (dist_proper['DisposalSite'] == j)][' Distance (km)'].empty else 0.0

#             # Get distance from disposal site to hub and convert to float
#             distance_to_hub = float(dist_proper[(dist_proper['PickupSector'] == hub_id) & (dist_proper['DisposalSite'] == j)][' Distance (km)'].iloc[0]) if not dist_proper[(dist_proper['PickupSector'] == hub_id) & (dist_proper['DisposalSite'] == j)][' Distance (km)'].empty else 0.0

#             # Get distance from hub to removal site i (assuming symmetric distances)
#             distance_from_hub_to_i = float(dist_sites[(dist_sites['Removal Site 1'] == hub_id) & (dist_sites['Removal Site 2'] == i)]['Distance'].iloc[0]) if not dist_sites[(dist_sites['Removal Site 1'] == hub_id) & (dist_sites['Removal Site 2'] == i)].empty else 0.0

#             # Total round-trip distance
#             total_distance = distance_from_hub_to_i + distance_to_disposal + distance_to_hub

#             # Store in dictionary
#             cost_distance_dict[(i, j)] = (disposal_cost, total_distance)

#     # Objective function with penalty for priority
#     penalty_factor = 100000
#     objective = gp.quicksum(
#         (Y[i] * cost_distance_dict[(i, j)][0] + Y[i]*(cost_distance_dict[(i, j)][1] * 0.1395 + 0.513)) * X[i, j]
#         for i in removal_ids for j in disposal_ids)

#     # Adding penalty logic for priorities
#     for i in removal_ids:
#         for k in removal_ids:
#             if i != k:
#                 priority_diff = float(removal_sites[removal_sites['NomSecteur'] == k]['Priority'].iloc[0]) - \
#                                 float(removal_sites[removal_sites['NomSecteur'] == i]['Priority'].iloc[0])
#                 objective += penalty_factor * Z[i, k] * max(0, priority_diff)

#     model.setObjective(objective, GRB.MINIMIZE)

#     # Constraints
#     # 1. Capacity of disposal sites
#     for j in disposal_ids:
#         model.addConstr(gp.quicksum(Y[i] * X[i, j] for i in removal_ids) <= float(disposal_sites[disposal_sites['NomDepot'] == j]['Capacite m3'].iloc[0]))

#     # 2. Completeness of snow removal
#     for i in removal_ids:
#         model.addConstr(gp.quicksum(X[i, j] for j in disposal_ids) == 1)

#     # 3. Total snow transported from each removal site equals its snow volume
#     for i in removal_ids:
#         total_snow_volume = float(removal_sites[removal_sites['NomSecteur'] == i]['Volume m3'].iloc[0])
#         model.addConstr(gp.quicksum(Y[i] * X[i, j] for j in disposal_ids) == total_snow_volume)

#     # Additional constraints for priority ordering
#     for i in removal_ids:
#         for k in removal_ids:
#             if i != k:
#                 model.addConstr(gp.quicksum(X[i, j] for j in disposal_ids) - gp.quicksum(X[k, j] for j in disposal_ids) <= Z[i, k])

#     # Optimize the model
#     model.optimize()

    
#     # Check if an optimal solution was found
#     if model.status == GRB.OPTIMAL:
#         # Store total cost for the hub
#         temp_results_df = pd.DataFrame({'Hub': [potential_hub], 'TotalCost': [model.ObjVal]})
#         results_df = pd.concat([results_df, temp_results_df], ignore_index=True)

#         # Store route details
#         for i in removal_ids:
#             for j in disposal_ids:
#                 if X[i, j].X > 0.5:  # If this route is used
#                     snow_transported = Y[i].X
#                     temp_route_details = pd.DataFrame({
#                         'Hub': [potential_hub],
#                         'RemovalSite': [i],
#                         'DisposalSite': [j],
#                         'SnowTransported': [snow_transported]
#                     })
#                     route_details_df = pd.concat([route_details_df, temp_route_details], ignore_index=True)
#     else:
#         print(f"No optimal solution found for hub {potential_hub}.")


## Exporting the optimal solution for each hub to a csv file

In [ ]:
# # Export the results and route details to CSV files
# results_df.to_csv('Optimal_Hub_Results.csv', index=False)
# route_details_df.to_csv('Route_Details.csv', index=False)

## Running model for 'VSP-206'

In [8]:
import gurobipy as gp
from gurobipy import GRB

# Create a Gurobi model
model = gp.Model('SnowRemoval')
model.Params.OutputFlag = 0

# Set the NonConvex parameter
model.setParam('NonConvex', 2)

# Define the hub (assuming the hub is one of the removal sites)
hub_id = 'VSP-206'  # Replace with the actual ID of the hub 

# Decision variables
removal_ids = removal_sites['NomSecteur'].tolist()
disposal_ids = disposal_sites['NomDepot'].tolist()
X = model.addVars(removal_ids, disposal_ids, vtype=GRB.BINARY, name="X")
Y = model.addVars(removal_ids, vtype=GRB.CONTINUOUS, name="Y")
Z = model.addVars(removal_ids, removal_ids, vtype=GRB.BINARY, name="Z")  # Auxiliary variables

# Pre-calculate cost and round-trip distances
cost_distance_dict = {}

for i in removal_ids:
    for j in disposal_ids:
        # Get disposal cost and convert to float
        disposal_cost = float(disposal_sites[disposal_sites['NomDepot'] == j]['Cost ($/m3)'].iloc[0]) if disposal_sites[disposal_sites['NomDepot'] == j]['Cost ($/m3)'].iloc[0] else 0.0

        # Get distance to disposal site and convert to float
        distance_to_disposal = float(dist_proper[(dist_proper['PickupSector'] == i) & (dist_proper['DisposalSite'] == j)][' Distance (km)'].iloc[0]) if not dist_proper[(dist_proper['PickupSector'] == i) & (dist_proper['DisposalSite'] == j)][' Distance (km)'].empty else 0.0
        
        # Get distance from disposal site to hub and convert to float
        distance_to_hub = float(dist_proper[(dist_proper['PickupSector'] == hub_id) & (dist_proper['DisposalSite'] == j)][' Distance (km)'].iloc[0]) if not dist_proper[(dist_proper['PickupSector'] == hub_id) & (dist_proper['DisposalSite'] == j)][' Distance (km)'].empty else 0.0

        # Get distance from hub to removal site i (assuming symmetric distances)
        distance_from_hub_to_i = float(dist_sites[(dist_sites['Removal Site 1'] == hub_id) & (dist_sites['Removal Site 2'] == i)]['Distance'].iloc[0]) if not dist_sites[(dist_sites['Removal Site 1'] == hub_id) & (dist_sites['Removal Site 2'] == i)].empty else 0.0

        # Total round-trip distance
        total_distance = distance_from_hub_to_i + distance_to_disposal + distance_to_hub

        # Store in dictionary
        cost_distance_dict[(i, j)] = (disposal_cost, total_distance)

# Objective function with penalty for priority
penalty_factor = 100000
objective = gp.quicksum(
    (Y[i] * cost_distance_dict[(i, j)][0] + Y[i]*(cost_distance_dict[(i, j)][1] * 0.1395 + 0.513)) * X[i, j]
    for i in removal_ids for j in disposal_ids)

# Adding penalty logic for priorities
for i in removal_ids:
    for k in removal_ids:
        if i != k:
            priority_diff = float(removal_sites[removal_sites['NomSecteur'] == k]['Priority'].iloc[0]) - \
                            float(removal_sites[removal_sites['NomSecteur'] == i]['Priority'].iloc[0])
            objective += penalty_factor * Z[i, k] * max(0, priority_diff)

model.setObjective(objective, GRB.MINIMIZE)

# Constraints
# 1. Capacity of disposal sites
for j in disposal_ids:
    model.addConstr(gp.quicksum(Y[i] * X[i, j] for i in removal_ids) <= float(disposal_sites[disposal_sites['NomDepot'] == j]['Capacite m3'].iloc[0]))

# 2. Completeness of snow removal
for i in removal_ids:
    model.addConstr(gp.quicksum(X[i, j] for j in disposal_ids) == 1)

# 3. Total snow transported from each removal site equals its snow volume
for i in removal_ids:
    total_snow_volume = float(removal_sites[removal_sites['NomSecteur'] == i]['Volume m3'].iloc[0])
    model.addConstr(gp.quicksum(Y[i] * X[i, j] for j in disposal_ids) == total_snow_volume)

# Additional constraints for priority ordering
for i in removal_ids:
    for k in removal_ids:
        if i != k:
            model.addConstr(gp.quicksum(X[i, j] for j in disposal_ids) - gp.quicksum(X[k, j] for j in disposal_ids) <= Z[i, k])

# Optimize the model
model.optimize()

# Check if an optimal solution was found
if model.status == GRB.OPTIMAL:
    # Print the total cost
    total_cost = model.ObjVal
    print(f"Total Cost of Snow Removal: ${total_cost:.2f}")

    # Print the amount of snow transported for each route
    print("\nSnow Transported for Each Route:")
    for i in removal_ids:
        for j in disposal_ids:
            if X[i, j].X > 0.5:  # If this route is used
                snow_transported = Y[i].X
                print(f"Removal site {i} -> Disposal site {j}: {snow_transported} m³")
else:
    print("No optimal solution found.")

Total Cost of Snow Removal: $42049854.45

Snow Transported for Each Route:
Removal site VMA-109 -> Disposal site Iberville (PMR): 93274.51 m³
Removal site VMA-307 -> Disposal site Iberville (PMR): 82691.13 m³
Removal site VMA-306 -> Disposal site Iberville (PMR): 96229.11 m³
Removal site VMA-111 -> Disposal site Iberville (PMR): 85847.51 m³
Removal site VMA-110 -> Disposal site Iberville (PMR): 127483.61 m³
Removal site PMR-101 -> Disposal site Iberville (PMR): 158592.4 m³
Removal site PMR-102 -> Disposal site Iberville (PMR): 144354.73 m³
Removal site S-O-104 -> Disposal site Iberville (PMR): 136883.4 m³
Removal site PMR-204 -> Disposal site Iberville (PMR): 80600.83 m³
Removal site MHM-109 -> Disposal site Iberville (PMR): 126720.19 m³
Removal site PMR-203 -> Disposal site Iberville (PMR): 97228.93 m³
Removal site RPP-201 -> Disposal site St-Michel - Robert (VSP): 116858.26 m³
Removal site MHM-209 -> Disposal site Iberville (PMR): 124186.67 m³
Removal site MHM-210 -> Disposal site Ib